# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import AdaBoostClassifier
import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IMANE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\IMANE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\IMANE\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('df', engine)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    """Tokenize text
    Args:
        text:string. Text to tokenize.
    Returns:
        clean_tokens:list. The tokenized text.
    """
  
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # use the word tokenizer to convert text into tokens
    tokens = word_tokenize(text)
    #initiate Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #Lemmatize and strip
    clean_tokens = []
    for tok in tokens:
    # Using lemmatization to strip all the words
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier.
    (reference:Udacity classroom)
    
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
model = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

model

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000274BF0F4AF0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [7]:
#Train pipeline
model.fit(X_train, Y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000274BF0F4AF0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = model.predict(X_test)
Y_pred.shape

(5244, 36)

In [9]:
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], Y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.60      0.24      0.34      1266
           1       0.79      0.95      0.86      3938
           2       0.25      0.07      0.12        40

    accuracy                           0.77      5244
   macro avg       0.55      0.42      0.44      5244
weighted avg       0.74      0.77      0.73      5244

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      4349
           1       0.78      0.53      0.63       895

    accuracy                           0.90      5244
   macro avg       0.85      0.75      0.79      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      524

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      4758
           1       0.73      0.49      0.59       486

    accuracy                           0.94      5244
   macro avg       0.84      0.74      0.78      5244
weighted avg       0.93      0.94      0.93      5244

fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5191
           1       0.60      0.17      0.26        53

    accuracy                           0.99      5244
   macro avg       0.80      0.58      0.63      5244
weighted avg       0.99      0.99      0.99      5244

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4766
           1       0.87      0.76      0.81       478

    accuracy                           0.97      5244
   macro avg       0.92      0.88      0.90      5244
weighted avg       0.97      0.97      0.97      5244

col

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
model.get_params()


{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x00000274BF0F4AF0>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('starting_verb', StartingVerbExtractor())])),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000274BF0F4AF0>)),
                                                 ('tfidf',
                                                  TfidfTransformer())])),
                                ('starting_ver

In [11]:
parameters = {
        'clf__estimator__n_estimators': [50, 100]
        }

model = GridSearchCV(model, param_grid=parameters)

In [12]:
# Train model
model.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000274BF0F4AF0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators':

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
model_predict_y = model.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], model_predict_y[:, i]))

related
              precision    recall  f1-score   support

           0       0.60      0.24      0.34      1266
           1       0.79      0.95      0.86      3938
           2       0.25      0.07      0.12        40

    accuracy                           0.77      5244
   macro avg       0.55      0.42      0.44      5244
weighted avg       0.74      0.77      0.73      5244

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      4349
           1       0.78      0.53      0.63       895

    accuracy                           0.90      5244
   macro avg       0.85      0.75      0.79      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      524

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      4758
           1       0.73      0.49      0.59       486

    accuracy                           0.94      5244
   macro avg       0.84      0.74      0.78      5244
weighted avg       0.93      0.94      0.93      5244

fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5191
           1       0.60      0.17      0.26        53

    accuracy                           0.99      5244
   macro avg       0.80      0.58      0.63      5244
weighted avg       0.99      0.99      0.99      5244

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4766
           1       0.87      0.76      0.81       478

    accuracy                           0.97      5244
   macro avg       0.92      0.88      0.90      5244
weighted avg       0.97      0.97      0.97      5244

col

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.